### 사전 준비 사항 

#### (1) uv add (터미널)

```bash
uv add rank_bm25
```

#### (2) .env 파일 세팅
```bash
OPENAI_API_KEY = ""
HF_TOKEN = ""
```

#### (3) pdf 파일 세팅
pdf 파일 100개를 `data/raw/files` 에 위치합니다.  
eval 파일(csv 2개, jsonl 1개)을 `data/raw/eval` 에 위치합니다.(*30개 파일 합친 버전)  

#### 실행 방법

1. 처음 1회(커널 새로 시작): 처음부터 끝까지 순서대로 실행
2. exp1 결과 저장 확인
3. <실험 ID 변경> 셀의 exp_id 를 변경 후 새로운 실험 진행 (실험 결과 저장 확인 후 커널 재시작)  
\*커널 재시작하지 않는 경우, 실험 ID 변경 + 실험 진행 섹션 코드만 실행해도 됨.(OOM 발생 가능성이 있어 권장하지 않음.)
4. 원하는 실험로 변경 및 반복

In [1]:
import json, re, unicodedata
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI
from sentence_transformers import SentenceTransformer

from preprocess.pp_basic import docs, BASE_DIR, EVAL_DIR, GOLD_EVIDENCE_CSV, GOLD_FIELDS_JSONL
from preprocess.rag_experiment import (
    CONFIG, ExperimentSpec, load_questions_df, make_components, RAGExperiment
)

load_dotenv(find_dotenv(), override=False)
client = OpenAI()

# embed 모델은 커널에서 1번만 로드(중요)
embed_model = SentenceTransformer("nlpai-lab/KoE5")

# gold 로드
gold_evidence_df = pd.read_csv(GOLD_EVIDENCE_CSV)

def load_gold_fields_jsonl(path):
    rows = []
    with open(path, "r", encoding="utf-8", errors="replace") as f:
        for line in f:
            if line.strip():
                rows.append(json.loads(line))
    out = []
    for r in rows:
        iid = r["instance_id"]
        doc_id = r.get("doc_id", "")
        fields = r.get("fields", {}) or {}
        for k, v in fields.items():
            out.append({"instance_id": iid, "doc_id": doc_id, "field": k, "gold": v})
    return pd.DataFrame(out)

gold_fields_df = load_gold_fields_jsonl(GOLD_FIELDS_JSONL)

questions_df = load_questions_df()

print("gold_evidence_df:", gold_evidence_df.shape)
print("gold_fields_df:", gold_fields_df.shape)
print("questions_df:", questions_df.shape)
print("n_docs:", len(docs))

d:\dev\github\codeit-part3-team4\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 391/391 [00:00<00:00, 574.68it/s, Materializing param=pooler.dense.weight]                               


gold_evidence_df: (630, 5)
gold_fields_df: (630, 4)
questions_df: (311, 5)
n_docs: 100


In [2]:
# 평가 문서 필터(커널당 1회)
def name_key(s: str) -> str:
    s = unicodedata.normalize("NFC", str(s)).strip()
    s = unicodedata.normalize("NFKC", s)
    s = re.sub(r"\s+", " ", s)
    return s

gold_doc_key_set = set(name_key(x) for x in gold_fields_df["doc_id"].astype(str).unique())
EVAL_DOCS = [p for p in docs if name_key(p.name) in gold_doc_key_set]

print("Eval docs:", len(EVAL_DOCS))

Eval docs: 30


### 실험 ID 변경

In [3]:
RUN_EXP_ID = 3   # 1~18
N_DOCS = 1       # 디버깅은 1~5 추천, 전체는 None 또는 아래 라인 변경

# N개 문서 또는 전체 문서 테스트
RUN_DOCS = EVAL_DOCS[:N_DOCS]
# RUN_DOCS = EVAL_DOCS

# exp table (18개)
SPECS = {
    1:  ("C1","R1","G1"),  2:  ("C1","R1","G2"),
    3:  ("C1","R2","G1"),  4:  ("C1","R2","G2"),
    5:  ("C1","R3","G1"),  6:  ("C1","R3","G2"),
    7:  ("C2","R1","G1"),  8:  ("C2","R1","G2"),
    9:  ("C2","R2","G1"),  10: ("C2","R2","G2"),
    11: ("C2","R3","G1"),  12: ("C2","R3","G2"),
    13: ("C3","R1","G1"),  14: ("C3","R1","G2"),
    15: ("C3","R2","G1"),  16: ("C3","R2","G2"),
    17: ("C3","R3","G1"),  18: ("C3","R3","G2"),
}

c, r, g = SPECS[RUN_EXP_ID]
spec = ExperimentSpec(exp_id=RUN_EXP_ID, chunker=c, retriever=r, generator=g)
print("Running spec:", spec)
print("RUN_DOCS:", len(RUN_DOCS))

# (선택) 실험 컨텍스트 cap 조정하고 싶은 경우 주석 해제 및 수정
# CONFIG["max_context_chars"] = 4000

Running spec: ExperimentSpec(exp_id=3, chunker='C1', retriever='R2', generator='G1')
RUN_DOCS: 1


### 실험 수행 및 결과 저장

In [4]:
chunker, retriever, generator = make_components(spec, embed_model=embed_model, client=client)
rag = RAGExperiment(chunker=chunker, retriever=retriever, generator=generator, questions_df=questions_df)

rows = []
for doc_path in tqdm(RUN_DOCS, desc=f"Exp {spec.exp_id} docs"):
    m = rag.run_single_doc_metrics(
        doc_path,
        gold_fields_df=gold_fields_df,
        gold_evidence_df=gold_evidence_df,
        top_k=CONFIG["top_k"],
        sim_threshold=80,
    )
    m["exp_id"] = spec.exp_id
    m["chunker"] = spec.chunker
    m["retriever"] = spec.retriever
    m["generator"] = spec.generator
    rows.append(m)

doc_df = pd.DataFrame(rows)

# exp-level average
avg = doc_df[["ret_recall","ret_mrr","gen_fill","gen_match","gen_sim"]].mean(numeric_only=True)
exp_df = pd.DataFrame([{
    "exp_id": spec.exp_id,
    "chunk": spec.chunker,
    "retriever": spec.retriever,
    "model": spec.generator,
    "n_docs": len(doc_df),
    **{k: float(avg[k]) for k in avg.index},
}])

display(exp_df.round(4))

out_dir = BASE_DIR / "outputs"
out_dir.mkdir(parents=True, exist_ok=True)

doc_out = out_dir / f"exp{spec.exp_id:02d}_doclevel.csv"
exp_out = out_dir / f"exp{spec.exp_id:02d}_explevel.csv"

doc_df.to_csv(doc_out, index=False, encoding="utf-8-sig")
exp_df.to_csv(exp_out, index=False, encoding="utf-8-sig")

print("Saved:", doc_out)
print("Saved:", exp_out)

Exp 3 docs: 100%|██████████| 1/1 [03:00<00:00, 180.84s/it]


,exp_id,chunk,retriever,model,n_docs,ret_recall,ret_mrr,gen_fill,gen_match,gen_sim
0,3,C1,R2,G1,1,0.4762,0.1491,1.0,0.0952,29.0402


Saved: d:\dev\github\codeit-part3-team4\outputs\exp03_doclevel.csv
Saved: d:\dev\github\codeit-part3-team4\outputs\exp03_explevel.csv


In [5]:
# 문서별 pred_map 저장 (옵션)
pred_dir = out_dir / f"exp{spec.exp_id:02d}_pred_maps"
pred_dir.mkdir(parents=True, exist_ok=True)

saved = 0
for _, row in doc_df.iterrows():
    doc_id = str(row["doc_id"])
    pred_map = row.get("pred_map", None)
    if isinstance(pred_map, dict):
        # 파일명 안전화
        safe = re.sub(r"[\\/:*?\"<>|]", "_", doc_id)
        path = pred_dir / f"{safe}.json"
        with open(path, "w", encoding="utf-8") as f:
            json.dump(pred_map, f, ensure_ascii=False, indent=2)
        saved += 1

print("Saved pred_maps:", saved, "->", pred_dir)

Saved pred_maps: 1 -> d:\dev\github\codeit-part3-team4\outputs\exp03_pred_maps


### 디버깅(선택)

In [6]:
DEBUG = True  # 필요할 때만 True

if DEBUG:
    print("last_debug:", getattr(rag.generator, "last_debug", None))
    raw = getattr(rag.generator, "last_raw_text", "")
    print("raw_text_len:", len((raw or "").strip()))
    print("raw_text_preview:\n", (raw or "")[:600])

last_debug: {'model': 'gpt-5-mini', 'n_questions': 21, 'context_len': 4000, 'max_context_chars': 4000, 'prompt_len': 5731, 'response_status': 'completed', 'output_tokens': 743, 'output_text_repr': '\'{\\n  "project_name": "2024년 「벤처확인종합관리시스템 기능 고도화」 용역사업",\\n  "agency": "NOT_FOUND",\\n  "purpose": "복수의결권주식, 스톡옵션, 성과조건부주식교부 기능 구축 및 고도화",\\n  "budget": "352,000,000원(부가가치세 포함)",\\n  "contract_type": "제한경쟁입찰(협상\'', 'exception': None}
raw_text_len: 1508
raw_text_preview:
 {
  "project_name": "2024년 「벤처확인종합관리시스템 기능 고도화」 용역사업",
  "agency": "NOT_FOUND",
  "purpose": "복수의결권주식, 스톡옵션, 성과조건부주식교부 기능 구축 및 고도화",
  "budget": "352,000,000원(부가가치세 포함)",
  "contract_type": "제한경쟁입찰(협상에 의한 계약)",
  "deadline": "NOT_FOUND",
  "duration": "계약일로부터 150일",
  "requirements_must": "사업수행계획서(품질보증계획서, 위험관리계획서 포함), 착수보고서(보안확약서, 보안서약서 포함), 분석 시스템 분석서, 요구사항 정의서, UI 정의서, 인터페이스 설계서, 데이터 주제영역 정의서, 논리/물리 정의서, 엔터티 정의서, 테이블 정의서, 인덱스 정의서, 데이터 이관 및 이행 계획서, 중간보고서, 개발 프로그램 소스코드; PM은 제안업체 직원으로 전기간 상주; 투입인력 교체 시 발주기관 사전 승인 및 최소 14일 

In [7]:
resp = client.responses.create(
    model="gpt-5-mini",
    input="OK 라고만 출력해.",
    max_output_tokens=20,
    reasoning={"effort": "minimal"},  # reasoning 최소화
)
print("status:", resp.status)
print("error:", resp.error)
print("output_tokens:", resp.usage.output_tokens)
print("output:", resp.output)

# 텍스트 추출(버전에 따라 다를 수 있어 둘 다 시도)
print("output_text attr:", getattr(resp, "output_text", None))
print("as dict keys:", list(resp.model_dump().keys()))

status: completed
error: None
output_tokens: 19
output: [ResponseReasoningItem(id='rs_0ac4f640af1e424a00698c536465508196a718480f218b4036', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseOutputMessage(id='msg_0ac4f640af1e424a00698c53648e0c8196b91289bc8c9fa2ea', content=[ResponseOutputText(annotations=[], text='OK', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')]
output_text attr: OK
as dict keys: ['id', 'created_at', 'error', 'incomplete_details', 'instructions', 'metadata', 'model', 'object', 'output', 'parallel_tool_calls', 'temperature', 'tool_choice', 'tools', 'top_p', 'background', 'completed_at', 'conversation', 'max_output_tokens', 'max_tool_calls', 'previous_response_id', 'prompt', 'prompt_cache_key', 'prompt_cache_retention', 'reasoning', 'safety_identifier', 'service_tier', 'status', 'text', 'top_logprobs', 'truncation', 'usage', 'user', 'billing', 'frequency_penalty', 'presence_penalty', 'sto